<a href="https://colab.research.google.com/github/louisdennington-design/decision-tree-dissertation/blob/main/llm_makes_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [6]:
# Set base parameters

model_name = "Qwen/Qwen2.5-7B-Instruct"

output_path = "PATH"

In [ ]:
"""
Focus is on instruction-following models from Hugging Face
With free licence (Apache)
Qwen seems to have been trained on producing JSON formats
Parameters are good balance between small and big
Shoul also check Llama offerings
"""

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "Give me a short introduction to large language model."

messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True)

model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
start = text.find("{")
end = text.rfind("}")

# Check openai prompt engineering guie

def construct_prompt:
    #
    return f"""
    You are making a recommendation from a NICE guideline into a structured JSON.

    RULES:
    - output must be valid JSON only
    - do not invent clinical information; use only what is present
    - necessity is defined by vocab like ...
    - actions are
    - more than one action?
    - unknown is null

    CONTEXT: Heading

    Produce JSON with exactly these fields:
    - action
    """

def run_llm_on_entity(tokenizer, model, entity,):

    prompt = construct_prompt(entity):
